В данном блокноте удаляю дубликаты session_id, причём те, у которых target_event = 0

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from warnings import filterwarnings

filterwarnings('ignore')

%store -r df

# для этого датафрейма 
#df = df.sample(frac=1).reset_index(drop=True)
#df = df.sample(n=50000).reset_index(drop=True)

res = df[~((df.duplicated(['session_id'])) & (df['target_event']==0) )]
for col in res.columns:
    res[col] = res[col].fillna('unknown')

## visit_date и visit_time
здесь делаю некоторые манипуляции с датой и временем: смотрю сколько было целевых действий за месяц и за час - оказывается, час и месяц влияют на целевое действие

In [ ]:
%%time
res['visit_time'] = pd.to_datetime(res['visit_time'])
res['visit_hour'] = res['visit_time'].apply(lambda x: int(x.hour))
vc = res[res['target_event']==1]['visit_hour'].value_counts()

ranges_hour = [0, 1000, 2000, 3000, 4000]
cats_hour = [1, 2, 3, 4]
df_vc = pd.DataFrame(vc)
df_vc['hour'] = df_vc.index
df_vc = df_vc.rename(columns={'visit_hour': 'count'})
df_vc.reset_index(drop=True)
df_vc['cat'] = pd.cut(df_vc['count'], ranges_hour, labels=cats_hour)

res['cat_hour'] = res['visit_hour'].apply(lambda x: df_vc.loc[df_vc['hour'], 'cat'][x])

res['visit_date'] = pd.to_datetime(res['visit_date'])
res['visit_month'] = res['visit_date'].apply(lambda x: int(x.month))

vcm = res[res['target_event']==1]['visit_month'].value_counts()

ranges_month = [0, 5000, 6000, 7000, 8000]
cats_month = [1, 2, 3, 4]
df_vcm = pd.DataFrame(vcm)
df_vcm['month'] = df_vcm.index
df_vcm = df_vcm.rename(columns={'visit_month': 'count'})
df_vcm.reset_index(drop=True)
df_vcm['cat'] = pd.cut(df_vcm['count'], ranges_month, labels=cats_month)
res['cat_month'] = res['visit_month'].apply(lambda x: df_vcm.loc[df_vcm['month'], 'cat'][x])

res = res.drop(['visit_date', 'visit_month', 'visit_time', 'visit_hour'], axis=1)

## UTM_*
### utm_source

In [ ]:
def foo(col_name):
    df0 = res[res['target_event']==0].groupby([col_name]).agg({'target_event': 'count'}).rename(columns={'target_event': 'zeros'})
    df1 = res[res['target_event']==1].groupby([col_name]).agg({'target_event': 'count'}).rename(columns={'target_event': 'ones'})
    df_part = pd.merge(df0, df1, left_index=True, right_index=True, how='outer')
    df_part['zeros'] = df_part['zeros'].fillna(0).astype(int)
    df_part['ones'] = df_part['ones'].fillna(0).astype(int)
    df_part['part'] = df_part.apply(lambda a: round(a['ones']/a['zeros'], 4), axis=1)
    return df_part.drop(['zeros', 'ones'], axis=1)

In [ ]:
# src0 = res[res['target_event']==0].groupby(['utm_source']).agg({'target_event': 'count'}).rename(columns={'target_event': 'zeros'})
# src1 = res[res['target_event']==1].groupby(['utm_source']).agg({'target_event': 'count'}).rename(columns={'target_event': 'ones'})
# src_full = pd.merge(src0, src1, left_index=True, right_index=True, how='outer')
# src_full['zeros'] = src_full['zeros'].fillna(0).astype(int)
# src_full['ones'] = src_full['ones'].fillna(0).astype(int)
# src_full['part'] = src_full.apply(lambda a: round(a['ones']/a['zeros'], 4), axis=1)
# src_full = src_full.drop(['zeros', 'ones'], axis=1)
# src_full.sort_values(by=['part'], ascending=False).head(30)

In [ ]:
def utm_source(x):
    return src.loc[x]

adv_soc_net = ['QxAxdyPLuQMEcrdZWdWb', 'MvfHsxITijuriZxsqZqt', 'ISrKoXQCxqqYvAZICvjs', 'IZEXUFLARCUMynmHNBGo', 'PlbkrSYoHuZBWfYjYnfw','gVRrcxiDQubJiljoTbGm']
res['adv_soc_net'] = res['utm_source'].apply(lambda x: 'from_adv' if x in adv_soc_net else 'other_source')
# res['adv_soc_net'] = res['utm_source'].apply(lambda x: 1 if x in adv_soc_net else 0)

src = foo('utm_source')
res['utm_source'] = res['utm_source'].apply(lambda y: utm_source(y))

### utm_medium

In [ ]:
def utm_medium(x):
    return med.loc[x]

utm_medium_type = ['organic', 'referral', '(none)']
res['utm_medium_type'] = res['utm_medium'].apply(lambda x: 'organic' if x in ['organic', 'referral', '(none)'] else 'paid_traffic')
# res['utm_medium_type'] = res['utm_medium'].apply(lambda x: 1 if x in utm_medium_type else 0)

med = foo('utm_medium')
res['utm_medium'] = res['utm_medium'].apply(lambda y: utm_medium(y))

### utm_adcontent

In [ ]:
def utm_adcontent(x):
    return adcontent.loc[x]
# 
adcontent = foo('utm_adcontent')
res['utm_adcontent'] = res['utm_adcontent'].apply(lambda y: utm_adcontent(y))

### utm_campaign

In [ ]:
def utm_campaign(x):
    return campaign.loc[x]

campaign = foo('utm_campaign')
res['utm_campaign'] = res['utm_campaign'].apply(lambda y: utm_campaign(y))

In [ ]:
res = res.drop(['session_id', 'client_id', 'device_model', 'visit_number', 'utm_keyword'], axis=1)
# res.head()

## DEVICE_*

### device_category

device_category - 3 категории, пропусков нет. Из преобразований - будет последующее onehot-кодирование

### device_os, device_brand, device_browser

восстановим ОС из других значений device_*

In [ ]:
best_brands = ['Samsung', 'Xiaomi', 'Huawei', 'POCO']

dev_types = ['mobile', 'tablet']

android_brands = ['Xiaomi', 'Samsung', 'Huawei', 'Vivo', 'LG',
       'BQ', 'OnePlus', 'itel', 'Asus', 'Sony', 'Android', 'OPPO',
       'Nokia', 'Realme', 'Alcatel', 'Infinix', 'Micromax', 'Blackview',
       'Oukitel', 'ZTE', 'Motorola', 'HOMTOM', 'Google', 'Lenovo',
       'China Phone', 'Neffos', 'Tecno', 'DOOGEE', 'Vsmart', 'Meizu', 'Umidigi', 'Leagoo', 'Highscreen', 'Ulefone', 'HTC',
       'Cubot', 'Symphony', 'Vestel', 'Guophone', 'Wileyfox', 'Vertex',
       'POCO', 'TCL', 'SonyEricsson', 'Nomu', 'Prestigio',
       'Philips', 'LeEco', 'DEXP', 'Hisense', 'Vernee', 'Tele2', 'CAT',
       'H96', 'Kyocera', 'Inoi', 'Acer', 'Kingplay', 'Sharp',
       'MTC', 'iNew', 'Karbonn', 'Wiko', 'Gionee', 'Digma',
       'Yota Devices', 'Kata', 'Poptel', 'Flylion', 'AGM', 'Teclast',
       'Fly', 'MLS', 'LeTV', 'Nuu', 'Celkon', 'Chuwi', 'Walton',
       'Haier', 'Coolpad', 'Xiaolajiao', 'Lava', 'General Mobile',
       'Black Fox', 'Condor', 'Razer', 'Microsoft', 'Fujitsu',
       'Alldocube', 'ZOJI', 'Amazon', 'PPTV', 'Jiake', 'Mito', 'Oysters',
       'NOA', 'Caterpillar', 'SenseIT', 'Verizon', 'Komu', 'BLU', 'RCA',
       'Cube', 'AT&T', 'Tonbux', 'Dark', 'Elephone', 'T-Mobile',
       'Smartfren', 'Billion', 'Gome', 'A1', 'MTS', 'Geotel', 'BLUBOO',
       'Ananda', 'MediaTek', 'KingSing', 'Ark', 'Essential',
       'Dragon Touch', 'E&L', 'Fero',  'Wigor', 'Tanix',
       'Mlais', 'InFocus', 'Smartisan', 'Beelink', 'Unihertz', 'Orbic',
       'Honeywell', 'Archos', 'RED', 'Sonim', 'Jiayu', 'Evertek',
       'Leegoog', 'QMobile', 'INUI', 'Artel', 'Irbis', 'China Mobile',
       'Flexymove', 'Olla', 'Krip', 'Star', 'UMI', 'Iris', 'Ellipsis',
       'LTC', 'Motive', 'Advan', 'M-HORSE', 'Winnovo', 'ThL', 'Nomi',
       'Wings Mobile', 'MXQ', 'How', 'Centric', 'Maze', 'Corn',
       'China TVBox', 'Gretel', 'Vertu', 'Evercoss', 'teXet', 'Qbex',
       'Panasonic', 'Tagital', 'Dell', 'Maxvi']

In [ ]:
%%time
def foo_os(x):
    if x['device_os']=='unknown':
        if x['device_brand'] in android_brands:
            return 'Android'
        elif (x['device_brand'] == 'Apple') and (x.device_category in dev_types):
            return 'iOS'
    elif x['device_os'] != x['device_os']:
        if (x['device_category'] in dev_types) and (x['device_brand']=='Apple'):
            return 'iOS'
        elif x['device_brand'] in android_brands:
            return 'Android'
    else:
        return x['device_os']
    return x['device_os']
    
res['device_os'] = res.apply(lambda x: foo_os(x), axis=1)

device_brand преобразуем в три категории: Apple - 3, Samsung, Xiaomi, Huawei и Poco - 2, остальные бренды - 1
признак device_browser отбросим - 

In [ ]:
def brand2cat(x):
    if x == 'Apple':
        return int(3)
    elif x in best_brands:
        return int(2)
    else:
        return int(1)
    return int(0)

res['cat_device_brand'] = res['device_brand'].apply(lambda x: brand2cat(x))
res = res.drop(['device_brand'], axis=1)

In [ ]:
def foo_browser(x):
    t = 0
    if x in ['Chrome', 'Safari', 'YaBrowser']:
        t = 2
    elif x in ['Safari (in-app)', 'Android Webview', 'Samsung Internet']:
        t = 1
    else:
        t = 0
    return t

res['cat_browser'] = res['device_browser'].apply(lambda x: foo_browser(x))

In [ ]:
res = res.drop(['device_browser', 'device_screen_resolution'], axis=1)

## GEO_*
### geo_country

In [ ]:
res['is_rus'] = res['geo_country'].apply(lambda x: 1 if x=='Russia' else 0)

### geo_city

In [ ]:
def geo_city(x):
    return city.loc[x]

city = foo('geo_city')
res['geo_city'] = res['geo_city'].apply(lambda y: geo_city(y))

In [ ]:
# res = res.drop(['geo_country', 'geo_city'], axis=1)

In [ ]:
def view_occupancy(df):
    length = len(df)
    dic = dict()
    for col in df.columns:
        percent_filled = (length-df[col].isna().sum())/length
        dic[col] = percent_filled

    dic = sorted(dic.items(), key=lambda x: x[1], reverse=True)
    for k, v in dic:
        print('{}: {:.3f}%'.format(k, v*100))

In [ ]:
view_occupancy(res)

In [ ]:
res.head()

In [ ]:
# res = res.sample(frac=1).reset_index(drop=True)
res = res.sample(n=200000).reset_index(drop=True)

cols_for_ohe = ['device_category', 'device_os', 'geo_country', 'adv_soc_net', 'utm_medium_type']

ohe = OneHotEncoder(sparse=False)

ohe.fit(res[cols_for_ohe])
transformed = ohe.transform(res[cols_for_ohe])
res[ohe.get_feature_names_out()] = transformed
df_ohed = res.drop(cols_for_ohe, axis=1)

%store df_ohed